# The Sparsifier Class

In this notebook we introduce the `Sparsifier` class and demonstrate its basic functionality

In [ ]:
import numpy as np
from sparseklearn import Sparsifier

In [ ]:
%load_ext autoreload
%autoreload 2

## Fitting the sparsifier

`fit_sparsifier` will apply the transform $HD$ to $X$, draw the `num_feat_comp` indices we will keep for each row of $HDX$, and then discard the data. As an example, we'll generate a dataset $X \in \mathbb{R}^{10 \times 5}$, where each row is a datapoint and there are 5 features. We'll sparsify $X$ into 3 dimensions, so that we are keeping $RHDX \in \mathbb{R}^{10 \times 3}$.

In [ ]:
rs = np.random.RandomState(12)
num_samp, num_feat_full = 10, 5
X = rs.rand(num_samp, num_feat_full)
num_feat_comp = 3
sp = Sparsifier(num_feat_full = num_feat_full, num_feat_comp = num_feat_comp, num_samp = num_samp, random_state = rs, transform = None)
sp.fit_sparsifier(X)

The `mask` attribute indicates which indices are kept for each datapoint:

In [ ]:
sp.mask

The `RHDX` attribute contains the data: 

In [ ]:
sp.RHDX

Since we didn't use a transform in the Sparsifier, `RHDX` is just a subset of `X`. Check this for an arbitrary datapoint:

In [ ]:
row = 3
(X[row][sp.mask[row]] == sp.RHDX[row]).all()

## Operations on sparsified data

The purpose of sparsifying the data is to permit operations on mixtures of sparse datapoints and dense statistics. Having fit the `Sparsifier` object, we can discard $X$ and use built-in methods to compute things like distances and covariances. We can compute the (approximate) pairwise distances between all the datapoints in the sample, for example:

In [ ]:
sp.pairwise_distances()

We can also compute the pairwise distances between the compressed data and a set of dense datapoints, which might be statistics we wish to estimate.

In [ ]:
dense_statistics = np.random.rand(4,num_feat_full)
sp.pairwise_distances(dense_statistics)

The `Sparsifier` class contains several other built-in methods to compute related quantities, including:
- `weighted_means`
- `weighted_means_and_variances`
- `pairwise_mahalanobis_distances`
See the docstrings for more information:

In [ ]:
help(sp.pairwise_mahalanobis_distances)

## The Johnson-Lindenstrauss Lemma

The Johnson-Lindenstrauss Lemma guarantees the existence of a projection $\Omega: \mathbb{R}^P \to \mathbb{R}^Q$ such that pairwise distances on a fixed number of points are preserved within a low tolerance with high probability. The Sparsifier is a random projection that acts like the JL projections. We will see an example of this here. We will first embed low-dimensional data into a higher dimension with some noise, and then see that as we keep more and more components, the error in the pairwise distances shrinks. 

In [ ]:
from scipy.stats import ortho_group
from scipy.spatial import distance
from sklearn.metrics import mean_squared_error
original_latent_dimension = 20
embedded_dimension = 100
num_samp = 100
rs = np.random.RandomState(73)
X = rs.rand(num_samp, original_latent_dimension) # generate the low-dimensional data
X = np.concatenate((X, np.zeros([num_samp,embedded_dimension-original_latent_dimension])), axis=1) # embed into higher dimension
X += rs.normal(scale = 1e-8, size = (num_samp, embedded_dimension)) # add noise
X = X[:, rs.permutation(np.arange(0,embedded_dimension,1))] # shuffle columns
true_distances = distance.squareform(distance.pdist(X))

In [ ]:
sp = Sparsifier(num_samp = num_samp, num_feat_full = embedded_dimension, num_feat_comp = 10)
sp.fit_sparsifier(X=X)
HDX = sp.apply_HD(X)

mse = np.zeros([3, len(num_feat_comp_array)])
num_feat_comp_array = np.arange(1,original_latent_dimension*3)
for i, num_feat_comp in enumerate(num_feat_comp_array):
    
    sp1 = Sparsifier(num_samp = num_samp, num_feat_full = embedded_dimension, num_feat_comp = num_feat_comp)
    sp1.fit_sparsifier(HDX=HDX)
    mse[0,i] = mean_squared_error(true_distances, sp1.pairwise_distances())
    
    sp2 = Sparsifier(num_samp = num_samp, num_feat_full = embedded_dimension, num_feat_comp = num_feat_comp, num_feat_shared = num_feat_comp//10 + 1)
    sp2.fit_sparsifier(HDX=HDX)
    mse[1,i] = mean_squared_error(true_distances, sp2.pairwise_distances())
    
    sp3 = Sparsifier(num_samp = num_samp, num_feat_full = embedded_dimension, num_feat_comp = num_feat_comp, num_feat_shared = num_feat_comp)
    sp3.fit_sparsifier(HDX=HDX)
    mse[2,i] = mean_squared_error(true_distances, sp3.pairwise_distances())

In [ ]:
import matplotlib.pyplot as plt
fig,ax = plt.subplots(1,1,figsize=(5,5))
ax.plot(num_feat_comp_array, mse[0], label = 'No shared features', linewidth = 2)
ax.plot(num_feat_comp_array, mse[1], label = '~10% shared features', linewidth = 2)
ax.plot(num_feat_comp_array, mse[2], label = '100% shared features', linewidth = 2)
ax.set_ylabel('MSE', fontsize = 12)
ax.set_xlabel(r'Number of features kept, $Q$', fontsize = 12)
ax.legend()
plt.show()